In [1]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5 
%pip install -qr requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 14185, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 14185 (delta 27), reused 41 (delta 16), pack-reused 14130
Receiving objects: 100% (14185/14185), 13.52 MiB | 18.45 MiB/s, done.
Resolving deltas: 100% (9744/9744), done.
/content/yolov5
     |████████████████████████████████| 182 kB 22.7 MB/s 
     |████████████████████████████████| 62 kB 586 kB/s 
     |████████████████████████████████| 1.6 MB 45.8 MB/s 


In [2]:
%mkdir /content/yolov5/fire
%cd /content/yolov5/fire

/content/yolov5/fire


In [3]:
!curl -L "https://universe.roboflow.com/ds/dsI5r46TPH?key=xjGIQJedpN" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: train/images/fire_589_jpg.rf.6b3ed6473ff56e8741f01be02ce3201b.jpg  
 extracting: train/images/fire_589_jpg.rf.7507a1a524b8458142eb84e42c01b186.jpg  
 extracting: train/images/fire_593_jpg.rf.077627dff52dfd82b476ffbd25f6fada.jpg  
 extracting: train/images/fire_593_jpg.rf.5023148e59fdced17fd82866469166bf.jpg  
 extracting: train/images/fire_593_jpg.rf.654804f4dd4d041396a29e01093840c4.jpg  
 extracting: train/images/fire_595_jpg.rf.08deb2eb5f75f843b0b9814e83160fc4.jpg  
 extracting: train/images/fire_595_jpg.rf.7bf376e4810aa3e8bef6ad091b7bd4eb.jpg  
 extracting: train/images/fire_595_jpg.rf.cd799940fce22b0b4150a29453a806fd.jpg  
 extracting: train/images/fire_596_jpg.rf.7a2228f763f995abce148e31cca9bc25.jpg  
 extracting: train/images/fire_596_jpg.rf.8d3f95a43054788ab98a5526f81195db.jpg  
 extracting: train/images/fire_598_jpg.rf.2e40f51c78f1116bab6f444640772488.jpg  
 extracting: train/images/fire_598_jpg.rf.f77e873558fffc55f504daae1afdbda

In [4]:
from glob import glob
train_img_list = glob('/content/yolov5/fire/train/images/*.jpg')
test_img_list = glob('/content/yolov5/fire/test/images/*.jpg')
valid_img_list = glob('/content/yolov5/fire/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

3571 174 345


In [5]:
import yaml
with open ('/content/yolov5/fire/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open ('/content/yolov5/fire/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

with open ('/content/yolov5/fire/valid.txt', 'w') as f:
  f.write('\n'.join(valid_img_list) + '\n')

In [6]:
%cat /content/yolov5/fire/data.yaml

train: ../train/images
val: ../valid/images

nc: 2
names: ['fire', 'object']

In [7]:
import yaml

with open ('/content/yolov5/fire/data.yaml', 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [8]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

In [9]:
%%writetemplate /content/yolov5/fire/data.yaml

train: ./fire/train/images
test: ./fire/test/images
val: ./fire/valid/images

nc: 2
names: ['fire', 'object']

In [10]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [11]:
%cat /content/yolov5/models/custom_yolov5s.yaml


# Parameters
nc: 2  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, Fa

In [12]:
%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 32 --epochs 200 --data ./fire/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name fire_results --cache

/content/yolov5
train: weights=, cfg=./models/custom_yolov5s.yaml, data=./fire/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=fire_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-7-g85f8379 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_